In [1]:
from deep.GCN import GCN
from utities import set_seed
import torch
from torch import nn
from torch_geometric.datasets import Planetoid
from tqdm import tqdm

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lr = 1e-2
epochs = 200
seed = 2022

In [3]:
dataset = Planetoid('Datasets', 'Cora')
data = dataset[0].to(device)

In [4]:
gcn = GCN(dataset.num_node_features, dataset.num_classes).to(device)
lossFunc = nn.NLLLoss().to(device)
optimizer = torch.optim.Adam(gcn.parameters(), lr=lr, weight_decay=5e-4)
set_seed(seed)

In [5]:
gcn.train()
for _ in tqdm(range(epochs), desc=">>> GCN train"):
    output = gcn(data)
    loss = lossFunc(output[data.train_mask], data.y[data.train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

>>> GCN train: 100%|██████████| 200/200 [02:10<00:00,  1.53it/s]


In [6]:
gcn.eval()
pred = gcn(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8080
